#### LDW experimental data

##### logs

5/15/2024 WL: template provided. TODO: write a brief intro to the data discussing the background, and the meaning of the variables.

Data source: https://users.nber.org/~rdehejia/data/.nswdata2.html

Ref. Causal Effects in Nonexperimental Studies: Reevaluating the Evaluation of Training Programs
Rajeev H. Dehejia, Sadek Wahba. Journal of the American Statistical Association,1999.

This is data on a job training program (the treatment) that was intended to raise
future earnings (the outcome).  Total number of examples is $2675$. The income is $\$1000$ in the year of 1978.


\begin{align*}
%\begin{array}{ll}
\hline \text { Variable } & \quad \text { Description } \\
\hline \text { age } &\quad  \text { Age in years } \\
\text { educ } &\quad  \text { Years of education } \\
\text { black } &\quad  1=\text { Black; } 0 \text { otherwise } \\
\text { hisp } & \quad 1=\text { Hispanic; } 0 \text { otherwise } \\
\text { married } &\quad  1=\text { married; } 0 \text { otherwise } \\
\text { nodegr } &\quad  1=\text { no degree; } 0 \text { otherwise } \\
\text { re74 } &\quad  1974 \text { income}\\
\text { re75 } &\quad  1975 \text { income}  \\
\text { re78 } &\quad  1978 \text { income} \\
\text { treat } &\quad  1=\text { received treatment; } 0 \text { otherwise } \\
\hline
%\end{array}
\end{align*}

In [4]:
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

import statsmodels.api as sm
from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from data_process import encode_dataframe

   Intercept  Sales  CompPrice  Income  Advertising  Population  Price  Age  \
0          1   9.50        138      73           11         276    120   42   
1          1  11.22        111      48           16         260     83   65   
2          1  10.06        113      35           10         269     80   59   
3          1   7.40        117     100            4         466     97   55   
4          1   4.15        141      64            3         340    128   38   

   Education  ShelveLoc_Bad  ShelveLoc_Medium  ShelveLoc_Good  Urban_No  \
0          0            1.0               0.0             0.0       0.0   
1          1            0.0               0.0             1.0       0.0   
2          1            0.0               1.0             0.0       0.0   
3          2            0.0               1.0             0.0       0.0   
4          0            1.0               0.0             0.0       1.0   

   Urban_Yes  US_No  US_Yes  
0        1.0    0.0     1.0  
1        1.0  

In [5]:
raw_data_all = pd.read_csv('./data/exp_LDW.csv').drop(["u74", "u75"], axis=1)

In [6]:
raw_data_all.head()

,T,age,educ,black,hisp,married,nodegr,re74,re75,re78
0,0,23,10,1,0,0,1,0.0,0.0,0.00
1,0,26,12,0,0,0,0,0.0,0.0,12383.68
2,0,22,9,1,0,0,1,0.0,0.0,0.00
3,0,18,9,1,0,0,1,0.0,0.0,10740.08
4,0,45,11,1,0,0,1,0.0,0.0,11796.47


In [7]:
# stats for treatment
print(raw_data_all.describe())
print()
print(sum(raw_data_all["T"] == 1), sum(raw_data_all["T"] == 0))
print()
print(sum(raw_data_all["re78"] > 0), sum(raw_data_all["re78"] == 0))

# about 7% enrolled in the training program
# about 13% unemployed in 1978

                T         age        educ       black       hisp     married  \
count  445.000000  445.000000  445.000000  445.000000  445.00000  445.000000   
mean     0.415730   25.370787   10.195506    0.833708    0.08764    0.168539   
std      0.493402    7.100282    1.792119    0.372762    0.28309    0.374766   
min      0.000000   17.000000    3.000000    0.000000    0.00000    0.000000   
25%      0.000000   20.000000    9.000000    1.000000    0.00000    0.000000   
50%      0.000000   24.000000   10.000000    1.000000    0.00000    0.000000   
75%      1.000000   28.000000   11.000000    1.000000    0.00000    0.000000   
max      1.000000   55.000000   16.000000    1.000000    1.00000    1.000000   

           nodegr          re74          re75          re78  
count  445.000000    445.000000    445.000000    445.000000  
mean     0.782022   2102.265311   1377.138368   5300.763699  
std      0.413337   5363.582400   3150.960771   6631.491695  
min      0.000000      0.000000

### Task: predict re78 using all other variables

In [8]:
#Split data into training features and labels
X, y = raw_data_all.loc[:, raw_data_all.columns != 're78'], raw_data_all['re78']

# np.bincount(X['T']) # array([260, 185])

In [9]:
# Compute the covariance matrix
covariance_matrix = X.cov()
print(covariance_matrix)

                 T          age        educ      black       hisp     married  \
T         0.243446     0.185596    0.062683   0.003973  -0.011742    0.008604   
age       0.185596    50.414010    0.292211   0.230717  -0.181218    0.558989   
educ      0.062683     0.292211    3.211691   0.030332  -0.073479    0.054813   
black     0.003973     0.230717    0.030332   0.138951  -0.073231    0.003315   
hisp     -0.011742    -0.181218   -0.073479  -0.073231   0.080140    0.000962   
married   0.008604     0.558989    0.054813   0.003315   0.000962    0.140449   
nodegr   -0.030798    -0.331157   -0.470802   0.006463   0.010138   -0.005972   
re74     -2.788176   -46.795663  395.918982  10.887865 -43.250703  286.480956   
re75     64.548727  1177.380020  146.001711 -66.231788  36.398415  316.371643   

             nodegr          re74          re75  
T         -0.030798 -2.788176e+00  6.454873e+01  
age       -0.331157 -4.679566e+01  1.177380e+03  
educ      -0.470802  3.959190e+02  1.46

In [10]:
# X_dat = encode_dataframe(X, add_intercept=True)
X_dat = X.copy() 
X_dat.insert(0, "Intercept", 1)

model = sm.OLS(y, X_dat)  # no intercept as default for sm.OLS
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     2.804
Date:                Wed, 23 Jul 2025   Prob (F-statistic):            0.00329
Time:                        11:22:41   Log-Likelihood:                -4534.2
No. Observations:                 445   AIC:                             9088.
Df Residuals:                     435   BIC:                             9129.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    785.0615   3374.969      0.233      0.8

In [11]:
# coefficients:
print("Coefficients:\n", results.params)
print()
# R-squared value:
print("R-squared:", results.rsquared)
print()
# standard errors:
print("Standard errors:\n", results.bse)
print()
# confidence intervals for the model parameters:
conf_intervals = results.conf_int()
print("Confidence intervals:\n", conf_intervals)
print()
# p-values of the parameters:
p_values = results.pvalues
print("p-values:\n", p_values)
print()

# Predicted values:
# predictions = results.predict(X)
# print("Predicted values:\n", predictions)
# print()
# Residuals:
# residuals = results.resid
# print("Residuals:\n", residuals)


Coefficients:
 Intercept     785.061481
T            1676.342644
age            55.316678
educ          395.734297
black       -2159.522141
hisp          164.032694
married      -138.725225
nodegr        -70.680676
re74            0.082141
re75            0.052764
dtype: float64

R-squared: 0.05482998592227473

Standard errors:
 Intercept    3374.968701
T             638.682016
age            45.283943
educ          227.414794
black        1169.035822
hisp         1549.457027
married       879.727925
nodegr       1004.387097
re74            0.077422
re75            0.135476
dtype: float64

Confidence intervals:
                      0            1
Intercept -5848.211450  7418.334411
T           421.056297  2931.628990
age         -33.685852   144.319207
educ        -51.234114   842.702708
black     -4457.183055   138.138773
hisp      -2881.320395  3209.385783
married   -1867.771017  1590.320567
nodegr    -2044.735647  1903.374295
re74         -0.070026     0.234308
re75         -0.2135

In [12]:
# calculate VIF for each feature in the array
# create a new DataFrame to store VIF values and feature names
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns

# compute VIF for each feature using list comprehension
vif_data["VIF"] = [VIF(X.values, i) for i, _ in enumerate(X.columns)]

print(vif_data)

   Feature        VIF
0        T   1.739837
1      age  12.318756
2     educ  14.709019
3    black  11.135342
4     hisp   2.004444
5  married   1.350245
6   nodegr   4.361578
7     re74   2.059179
8     re75   2.261413


In [13]:
# calculate VIF for each feature in the array
# rreate a new DataFrame to store VIF values and feature names
vif_data = pd.DataFrame()
vif_data["Feature"] = X_dat.columns

# compute VIF for each feature using list comprehension
vif_data["VIF"] = [VIF(X_dat.values, i) for i, _ in enumerate(X_dat.columns)]

print(vif_data)

     Feature         VIF
0  Intercept  119.473999
1          T    1.039270
2        age    1.081924
3       educ    1.738314
4      black    1.987354
5       hisp    2.013555
6    married    1.137558
7     nodegr    1.803712
8       re74    1.804644
9       re75    1.907069
